In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# data_100k = pd.read_csv('blackjack_simulator.csv').sample(n=100000, random_state=21)
data_500k = pd.read_csv('blackjack_simulator.csv').sample(n=500000, random_state=21)
# data_1m = pd.read_csv('blackjack_simulator.csv').sample(n=1000000, random_state=21)

# data_100k.to_csv('./blackjack_sample_100k.csv')
data_500k.to_csv('./blackjack_sample_500k.csv')
# data_1m.to_csv('./blackjack_sample_1m.csv')

data_500k.head()

,shoe_id,cards_remaining,dealer_up,initial_hand,dealer_final,dealer_final_value,player_final,player_final_value,actions_taken,run_count,true_count,win
33791619,556105,393,4,"[4, 7]","[4, 3, 10]",17,"[[4, 7, 10]]",[21],[['D']],-5,0,2.0
19602821,322598,389,9,"[2, 10]","[9, 10]",19,"[[2, 10, 7]]",[19],"[['H', 'S']]",3,0,0.0
34664548,570473,268,10,"[7, 6]","[10, 4, 10]",24,"[[7, 6, 2, 10]]",[25],"[['H', 'H']]",15,2,-1.0
37395938,615427,246,10,"[4, 9]","[10, 10]",20,"[[4, 9, 11, 9]]",[23],"[['H', 'H']]",-2,0,-1.0
10736666,176685,136,7,"[10, 9]","[7, 10]",17,"[[10, 9]]",[19],[['S']],6,2,1.0


In [4]:
data2 = pd.read_excel('./blackjack cleaner.xlsx', sheet_name='blackjack_sample_500k')


data2.describe()

,dealer_up,player_card_1,player_card_2,player_card_3,player_card_4,player_card_5,win/loss
count,487743.000000,487743.000000,487743.000000,221187.000000,61392.000000,11360.000000,487743.000000
mean,7.345348,7.310001,7.308341,7.311515,7.337161,7.307570,0.503191
std,2.913487,2.915530,2.921654,2.921636,2.905504,2.902278,0.499990
min,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000
25%,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,0.000000
50%,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,1.000000
75%,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000
max,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,1.000000


In [5]:
data2.head()

,dealer_up,player_card_1,player_card_2,player_card_3,player_card_4,player_card_5,player_move_1,player_move_2,player_move_3,player_move_4,win/loss
0,4,4,7,10.0,NaN,NaN,D,NaN,NaN,NaN,1
1,9,2,10,7.0,NaN,NaN,H,S,NaN,NaN,1
2,10,7,6,2.0,10.0,NaN,H,H,NaN,NaN,0
3,10,4,9,11.0,9.0,NaN,H,H,NaN,NaN,0
4,7,10,9,NaN,NaN,NaN,S,NaN,NaN,NaN,1


In [6]:
# data2.columns


model_1_data = data2[['dealer_up', 'player_card_1', 'player_card_2', 'player_move_1', 'win/loss']].dropna(axis=0)
model_2_data = data2[['dealer_up', 'player_card_1', 'player_card_2', 'player_card_3', 'player_move_1', 'player_move_2', 'win/loss']].dropna(axis=0)
model_3_data = data2[['dealer_up', 'player_card_1', 'player_card_2', 'player_card_3', 'player_card_4', 'player_move_1', 'player_move_2', 'player_move_3', 'win/loss']].dropna(axis=0)

#['dealer_up', 'player_card_1', 'player_card_2', 'player_card_3',
#        'player_card_4', 'player_card_5', 'player_move_1', 'player_move_2',
#        'player_move_3', 'player_move_4', 'win/loss']

In [7]:
#getdummies
model_1_data = pd.get_dummies(model_1_data, columns=['dealer_up', 'player_card_1', 'player_card_2', 'player_move_1'])
model_2_data = pd.get_dummies(model_2_data, columns=['dealer_up', 'player_card_1', 'player_card_2', 'player_card_3', 'player_move_1', 'player_move_2'])
model_3_data = pd.get_dummies(model_3_data, columns=['dealer_up', 'player_card_1', 'player_card_2', 'player_card_3', 'player_card_4', 'player_move_1', 'player_move_2', 'player_move_3'])


In [8]:
#create X and Y variables for Random Forest
X1 = model_1_data.drop('win/loss', axis=1)
X2 = model_2_data.drop('win/loss', axis=1)
X3 = model_3_data.drop('win/loss', axis=1)

y1 = model_1_data['win/loss']
y2 = model_2_data['win/loss']
y3 = model_3_data['win/loss']

In [9]:
#train test split data fror all 3 random forest models
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=21)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=21)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, random_state=21)

In [10]:
#initialize all 3 random forest models & hyperperameter tuning.

#first iteration acc score = 70%
# rf_model_1 = RandomForestClassifier(n_estimators=500, random_state=21)
# rf_model_2 = RandomForestClassifier(n_estimators=500, random_state=21)
# rf_model_3 = RandomForestClassifier(n_estimators=500, random_state=21)

#second iteration acc score = 64%
# rf_model_1 = RandomForestClassifier(n_estimators=500, max_depth=4, random_state=21)
# rf_model_2 = RandomForestClassifier(n_estimators=500, max_depth=4, random_state=21)
# rf_model_3 = RandomForestClassifier(n_estimators=500, max_depth=4, random_state=21)

#3rd acc score = 64%
# rf_model_1 = RandomForestClassifier(n_estimators=1000, max_depth=4, criterion='gini', random_state=21)
# rf_model_2 = RandomForestClassifier(n_estimators=1000, max_depth=4, criterion='gini', random_state=21)
# rf_model_3 = RandomForestClassifier(n_estimators=1000, max_depth=4, criterion='gini', random_state=21)

#4th 0.6751728831788658 0.6514534717046041 0.700695383426695
# rf_model_1 = RandomForestClassifier(n_estimators=1000, max_depth=10, criterion='gini', random_state=21)
# rf_model_2 = RandomForestClassifier(n_estimators=1000, max_depth=10, criterion='gini', random_state=21)
# rf_model_3 = RandomForestClassifier(n_estimators=1000, max_depth=10, criterion='gini', random_state=21)

#5th 0.6750972599171505 0.6513674674617281 0.7005022213637242
# rf_model_1 = RandomForestClassifier(n_estimators=1200, max_depth=10, criterion='entropy', max_features='auto', random_state=21)
# rf_model_2 = RandomForestClassifier(n_estimators=1200, max_depth=10, criterion='entropy', max_features='auto', random_state=21)
# rf_model_3 = RandomForestClassifier(n_estimators=1200, max_depth=10, criterion='entropy', max_features='auto', random_state=21)

#6th 0.674971221147625 0.6511381228140588 0.700695383426695
# rf_model_1 = RandomForestClassifier(n_estimators=2500, max_depth=10, criterion='entropy', max_features='sqrt', random_state=21)
# rf_model_2 = RandomForestClassifier(n_estimators=2500, max_depth=10, criterion='entropy', max_features='sqrt', random_state=21)
# rf_model_3 = RandomForestClassifier(n_estimators=2500, max_depth=10, criterion='entropy', max_features='sqrt', random_state=21)

#6th 0.6932384401441883 0.6934522103090419 0.7095808383233533
# rf_model_1 = RandomForestClassifier(n_estimators=500, criterion='entropy', max_features='sqrt', random_state=21)
# rf_model_2 = RandomForestClassifier(n_estimators=500, criterion='entropy', max_features='sqrt', random_state=21)
# rf_model_3 = RandomForestClassifier(n_estimators=500, criterion='entropy', max_features='sqrt', random_state=21)

# #7th 0.6932888556519986 0.6937388911186285 0.7102569055437512 - winner
# rf_model_1 = RandomForestClassifier(n_estimators=1500, criterion='entropy', max_features='sqrt', random_state=21)
# rf_model_2 = RandomForestClassifier(n_estimators=1500, criterion='entropy', max_features='sqrt', random_state=21)
# rf_model_3 = RandomForestClassifier(n_estimators=1500, criterion='entropy', max_features='sqrt', random_state=21)

#testing
rf_model_1 = RandomForestClassifier(n_estimators=500, criterion='entropy', max_features='sqrt', random_state=21)
rf_model_2 = RandomForestClassifier(n_estimators=500, criterion='entropy', max_features='sqrt', random_state=21)
rf_model_3 = RandomForestClassifier(n_estimators=500, criterion='entropy', max_features='sqrt', random_state=21)


In [11]:
#fit all 3 random forest models ~ 5 mins
rf_model_1 = rf_model_1.fit(X1_train, y1_train)
rf_model_2 = rf_model_2.fit(X2_train, y2_train)
rf_model_3 = rf_model_3.fit(X3_train, y3_train)

In [58]:
#predict moves for all 3 models
predictions_1 = rf_model_1.predict(X1_test)
predictions_2 = rf_model_2.predict(X2_test)
predictions_3 = rf_model_3.predict(X3_test)

In [59]:
move1_cols = X1_test.columns

In [60]:
move2_cols = X2_test.columns

In [61]:
move3_cols = X3_test.columns

In [62]:
acc_score_1 = accuracy_score(y1_test, predictions_1)
acc_score_2 = accuracy_score(y2_test, predictions_2)
acc_score_3 = accuracy_score(y3_test, predictions_3)

print(acc_score_1, acc_score_2, acc_score_3)

0.6932384401441883 0.6934522103090419 0.7095808383233533


In [63]:
#classification report 1
cr1 = classification_report(y1_test, predictions_1)
print(cr1)

              precision    recall  f1-score   support

           0       0.66      0.76      0.71     57807
           1       0.74      0.63      0.68     61204

    accuracy                           0.69    119011
   macro avg       0.70      0.70      0.69    119011
weighted avg       0.70      0.69      0.69    119011



In [64]:
#classification report 2
cr2 = classification_report(y2_test, predictions_2)
print(cr2)

              precision    recall  f1-score   support

           0       0.66      0.75      0.70     16679
           1       0.74      0.64      0.68     18203

    accuracy                           0.69     34882
   macro avg       0.70      0.70      0.69     34882
weighted avg       0.70      0.69      0.69     34882



In [65]:
#classification report 3
cr3 = classification_report(y3_test, predictions_3)
print(cr3)

              precision    recall  f1-score   support

           0       0.66      0.69      0.68      4557
           1       0.75      0.73      0.74      5797

    accuracy                           0.71     10354
   macro avg       0.71      0.71      0.71     10354
weighted avg       0.71      0.71      0.71     10354



In [66]:
#play an audio notification when the cells are finished running
import winsound
winsound.Beep(500,300)
winsound.Beep(500,300)
winsound.Beep(500,300)


In [88]:
#testing code
import numpy as np

move_2_data = pd.DataFrame(data=np.reshape([0]*45, [1,45]), columns=move2_cols)

move_2_data.loc[0,'dealer_up_2'] = 1

move_2_data.columns

Index(['dealer_up_2', 'dealer_up_3', 'dealer_up_4', 'dealer_up_5',
       'dealer_up_6', 'dealer_up_7', 'dealer_up_8', 'dealer_up_9',
       'dealer_up_10', 'dealer_up_11', 'player_card_1_2', 'player_card_1_3',
       'player_card_1_4', 'player_card_1_5', 'player_card_1_6',
       'player_card_1_7', 'player_card_1_8', 'player_card_1_9',
       'player_card_1_10', 'player_card_1_11', 'player_card_2_2',
       'player_card_2_3', 'player_card_2_4', 'player_card_2_5',
       'player_card_2_6', 'player_card_2_7', 'player_card_2_8',
       'player_card_2_9', 'player_card_2_10', 'player_card_2_11',
       'player_card_3_2.0', 'player_card_3_3.0', 'player_card_3_4.0',
       'player_card_3_5.0', 'player_card_3_6.0', 'player_card_3_7.0',
       'player_card_3_8.0', 'player_card_3_9.0', 'player_card_3_10.0',
       'player_card_3_11.0', 'player_move_1_H', 'player_move_1_N',
       'player_move_2_D', 'player_move_2_H', 'player_move_2_S'],
      dtype='object')

In [86]:
#blackjack game

#import numpy as np

moves = ['H','D','S','R'] #removed P (split)
player_cards = []
dealer_cards = []
game_winner = 'unknown'

def get_random_card():

    #ace == 11
    #k, q, j == 10
    cards = [2,3,4,5,6,7,8,9,10,10,10,10,11]
    card = np.random.choice(cards)

    return card

def predict_move_1(player_card_1, player_card_2, dealer_card, move_1):

    outcome = 'loss'

    #encode the game state to be able to pass into prediction model
    move_1_data = pd.DataFrame(data=np.reshape([0]*35, [1,35]), columns=move1_cols)

    #player_card_1_x
    #player_card_2_x
    #player_move_1_x
    #dealer_up_x

    a = f'player_card_1_{player_card_1}'
    b = f'player_card_2_{player_card_2}'
    c = f'player_move_1_{move_1}'
    d = f'dealer_up_{dealer_card}'

    move_1_data.loc[0,a] = 1
    move_1_data.loc[0,b] = 1
    move_1_data.loc[0,c] = 1
    move_1_data.loc[0,d] = 1

    if(rf_model_1.predict(move_1_data)) == 1:
        outcome = 'win'

    #predict the outcome of the game with the defined parameters

    #if the outcome is win, return win, else return loss

    return outcome

def predict_move_2(player_card_1, player_card_2, dealer_card_1, player_card_3, move_1, move_2):

    outcome = 'loss'

    #encode input data fro move 2
    move_2_data = pd.DataFrame(data=np.reshape([0]*45, [1,45]), columns=move2_cols)

    #player_card_1_x
    #player_card_2_x
    #player_move_1_x
    #dealer_up_x
    #player_card_3_x
    #player_move_2_x

    a = f'player_card_1_{player_card_1}'
    b = f'player_card_2_{player_card_2}'
    c = f'player_move_1_{move_1}'
    d = f'dealer_up_{dealer_card_1}'
    e = f'player_card_3_{player_card_3}.0'
    f = f'player_move_2_{move_2}'

    move_2_data.loc[0,a] = 1
    move_2_data.loc[0,b] = 1
    move_2_data.loc[0,c] = 1
    move_2_data.loc[0,d] = 1
    move_2_data.loc[0,e] = 1
    move_2_data.loc[0,f] = 1

    if(rf_model_2.predict(move_2_data)) == 1:
        outcome = 'win'

    return outcome

def predict_move_3(player_card_1, player_card_2, dealer_card_1, player_card_3, player_card_4, move_1, move_2, move_3):

    outcome = 'loss'

    #encode input data fro move 3
    move_3_data = pd.DataFrame(data=np.reshape([0]*55, [1,55]), columns=move3_cols)

    #player_card_1_x
    #player_card_2_x
    #player_move_1_x
    #dealer_up_x
    #player_card_3_x
    #player_move_2_x
    #player_card_4_x
    #player_move_3_x

    a = f'player_card_1_{player_card_1}'
    b = f'player_card_2_{player_card_2}'
    c = f'player_move_1_{move_1}'
    d = f'dealer_up_{dealer_card_1}'
    e = f'player_card_3_{player_card_3}.0'
    f = f'player_move_2_{move_2}'
    g = f'player_card_4_{player_card_4}.0'
    h = f'player_move_3_{move_3}'

    move_3_data.loc[0,a] = 1
    move_3_data.loc[0,b] = 1
    move_3_data.loc[0,c] = 1
    move_3_data.loc[0,d] = 1
    move_3_data.loc[0,e] = 1
    move_3_data.loc[0,f] = 1
    move_3_data.loc[0,g] = 1
    move_3_data.loc[0,h] = 1

    if(rf_model_3.predict(move_3_data)) == 1:
        outcome = 'win'

    return outcome

def dealer_moves():
    
    #Hit or stay
    proceed = True

    while proceed == True:

        dealer_value = sum(dealer_cards)

        if dealer_value <= 16:
            #hit less than or equal to 16
            dealer_cards.append(get_random_card())
        else:
            #stay > 16
            proceed = False

    #Hit when card_values <=16 or (A and 6) else stay

    return True

def select_winner(dealer_total, player_total):
    winner = 'Dealer'

    if abs(21 - player_total) < abs(21 - dealer_total):
        winner = 'Player'

    return winner


for turn in range(1,4+1):

    #every turn, evaluate each of the possible moves for win / loss, using the correct model.
    if turn == 1:

        winning_moves_1 = []

        #generate initial state of card game:
        #2 player cards
        #1 dealer up card
        player_card_1 = get_random_card()
        player_card_2 = get_random_card()
        dealer_card   = get_random_card()
        dealer_card_2 = get_random_card() #not revealed until player is done

        #append the cards to a list to keep track of scores
        player_cards.append(player_card_1)
        player_cards.append(player_card_2)
        dealer_cards.append(dealer_card)
        dealer_cards.append(dealer_card_2)

        
        for move in moves:

            outcome = predict_move_1(player_card_1, player_card_2, dealer_card ,move)

            if outcome == 'win':
                winning_moves_1.append(move)
        
        print(f'Your Cards: {player_card_1}, {player_card_2}')
        print(f"Dealer's Card: {dealer_card}")
        print('Winning Moves: ',winning_moves_1)

        #player choses a move.
        move_1 = input('Enter Your Move (H = Hit, S = Stand, D = Double, R = Surrender): ')
        print(f'Player move = {move_1}')

        #if player stays or doubles, the dealer takes their turn
        if(move_1 == 'S' or move_1 == 'R'):
            break
            #dealter's turn, break out of turns

        if(move_1 == 'D'):
            player_cards.append(get_random_card())
            break

        if(move_1 == 'R'):
            #player loss due to surrender
            game_winner = 'dealer'
            print(f"Dealer's final cards = {dealer_cards}, value = {dealer_value}")
            print(f"Player's final cards = {player_cards}, value = {player_value}")
            print(f'Winner = {game_winner}')
            break #exit moves loop upon player loss

    elif turn == 2:

        winning_moves_2 = []

        #get new cards, if pplayer's count is> 21 then end game as loss
        #dealer does not get new cards until player is done

        player_card_3 = get_random_card()
        player_cards.append(player_card_3)

        player_value = sum(player_cards)
        dealer_value = sum(dealer_cards)

        if(player_value > 21):
            game_winner = 'dealer'
            print(f"Dealer's final cards = {dealer_cards}, value = {dealer_value}")
            print(f"Player's final cards = {player_cards}, value = {player_value}")
            print(f'Winner = {game_winner}')
            break #exit moves loop upon player loss

        for move in moves:

            outcome = predict_move_2(player_card_1, player_card_2, player_card_3, dealer_card, move_1=move_1, move_2=move)

            if outcome == 'win':
                winning_moves_2.append(move)
        
        print(f'Your Cards: {player_card_1}, {player_card_2}, {player_card_3}')
        print(f"Dealer's Card: {dealer_card}")
        print('Winning Moves: ',winning_moves_2)

        move_2 = input('Enter Your Move (H = Hit, S = Stand, D = Double, R = Surrender): ')
        print(f'Player move = {move_2}')

        if(move_2 == 'S' or move_2 == 'R'):
            break
            #dealter's turn, break out of turns

        if(move_2 == 'D'):
            player_cards.append(get_random_card())
            break

        if(move_2 == 'R'):
            #player loss due to surrender
            game_winner = 'dealer'
            print(f"Dealer's final cards = {dealer_cards}, value = {dealer_value}")
            print(f"Player's final cards = {player_cards}, value = {player_value}")
            print(f'Winner = {game_winner}')
            break #exit moves loop upon player loss
    
    elif turn == 3:
        #get new cards, if pplayer's count is> 21 then end game as loss

        winning_moves_3 = []

        player_card_4 = get_random_card()
        player_cards.append(player_card_4)

        player_value = sum(player_cards)
        dealer_value = sum(dealer_cards)

        if(player_value > 21):
            game_winner = 'dealer'
            print(f"Dealer's final cards = {dealer_cards}, value = {dealer_value}")
            print(f"Player's final cards = {player_cards}, value = {player_value}")
            print(f'Winner = {game_winner}')
            break #exit moves loop upon player loss

        for move in moves:

            outcome = predict_move_3(player_card_1, player_card_2, player_card_3, player_card_4, dealer_card, move_1=move_1, move_2=move_2, move_3=move)

            if outcome == 'win':
                winning_moves_3.append(move)
        
        print(f'Your Cards: {player_card_1}, {player_card_2}, {player_card_3}, {player_card_4}')
        print(f"Dealer's Card: {dealer_card}")
        print('Winning Moves: ',winning_moves_3)

        move_3 = input('Enter Your Move (H = Hit, S = Stand, D = Double, R = Surrender): ')
        print(f'Player move = {move_3}')

        if(move_3 == 'S'):
            break
            #dealter's turn, break out of turns

        if(move_3 == 'R'):
            #player loss due to surrender
            game_winner = 'dealer'
            print(f"Dealer's final cards = {dealer_cards}, value = {dealer_value}")
            print(f"Player's final cards = {player_cards}, value = {player_value}")
            print(f'Winner = {game_winner}')
            break #exit moves loop upon player loss
        
        if(move_3 == 'D'):
            player_cards.append(get_random_card())
            break

    elif turn == 4:
        print('Joe Pesci Opened the door With your head')
    
    else:
        print('turn out of bounds')

#code only runs if the loop is broken, if the game contunes for more than 3 moves, 
#joe pesci opens the door with your head

if(game_winner == 'unknown'):

    dealer_moves()

    dealer_total = sum(dealer_cards)
    player_total = sum(player_cards)

    #if dealer > 21 then player wins
    if(dealer_total > 21):
        game_winner = 'player'
    else:
        game_winner = select_winner(dealer_total, player_total)

    print(f"Dealer's final cards = {dealer_cards}, value = {dealer_total}")
    print(f"Player's final cards = {player_cards}, value = {player_total}")
    print(f'Winner = {game_winner}')

else:
    pass

Your Cards: 4, 7
Dealer's Card: 5
Winning Moves:  ['D']
Player move = R


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- player_move_1_R
